### Cox analysis for RCT

In [1]:
print('Cox analysis for RCT')
print('Parse an events.csv file and output a summary for each agent in the trial')
print('Inputs: events_fpath variable')

Cox analysis for RCT
Parse an events.csv file and output a summary for each agent in the trial
Inputs: events_fpath variable


In [2]:
import sys, csv, os
import numpy as np
import pandas as pd
import pdb

In [14]:
#examples
#paths = {'trial1':'',
#         'trial2':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\draft2\events.csv",
#         'trial8444':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604889008444_64pc\instance_1\output\events.csv",
#         'trial4':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1604882689569_35pc\instance_1\output\events.csv",
#         'trial85a':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates85pc\simphony_model_1614483337038\instance_1\output\events.csv",
#         'trial85b':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates85pc\simphony_model_1614485140334\instance_1\output\events.csv",
#         'trial85c':r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates85pc\simphony_model_1614526877900\instance_1\output\events.csv"
#        }
#toy trial
events_fpath = r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1661546710721_2weekly\instance_1\output\events.csv"
#events_fpath = r"C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates85pc\simphony_model_1614483337038\instance_1\output\events.csv"
chosen_trial = 'trial721'  #wishlist: extract from dirname
#D2E60
#events_fpath = paths[chosen_trial]
#chosen_trial=''

In [15]:
print("Reading events from: " + events_fpath)

Reading events from: C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1661546710721_2weekly\instance_1\output\events.csv


In [16]:
time_between_bleeds = 14 #days
#warning: this must have the same value as vaccine_followup_weeks*7

In [17]:
reporting_frequency = time_between_bleeds/365
#rebase_time_to_final_dose

In [18]:
print(f"Time between simulated bleeds (days): {time_between_bleeds}")

Time between simulated bleeds (days): 14


In [19]:
vaccine_trial_end_date = 1400/365.0 #external default

In [20]:
print(f"vaccine_trial_end_date (years): {vaccine_trial_end_date}")

vaccine_trial_end_date (years): 3.835616438356164


In [21]:
output_fpath = f'{events_fpath}_history_{chosen_trial}_gap{time_between_bleeds}.csv'
print(f"Output file: {output_fpath}")

Output file: C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1661546710721_2weekly\instance_1\output\events.csv_history_trial721_gap14.csv


In [22]:
#events_file = open(events_file,'r')
#events = csv.DictReader(events_file)

In [23]:
#events_file.close()

In [24]:
trial_header = pd.read_csv(events_fpath, nrows=2, header=1)

In [25]:
#trial_header

In [26]:
burn_in_days = trial_header.burn_in_days[0]
vaccine_schedule = trial_header.vaccine_schedule[0]
vaccine_enrollment_launch_day = trial_header.vaccine_enrollment_launch_day


In [27]:
#vaccine_trial_end_date = 2*vaccine_trial_end_date

In [28]:
print(f'vaccine_schedule: {vaccine_schedule}')

vaccine_schedule: D2E60


In [29]:
events = pd.read_csv(events_fpath, header=4)
events.rename(columns={'Num Buddies':'total_network_size'}, inplace=True)

In [30]:
#events.tail()

In [31]:
#events[events.Event == 'enteredfollowup'].head()

In [32]:
#pd.unique(events.HCV)

In [33]:
trial_starts = events[events.Event == 'trialstarted'].copy()
trial_starts['RNA_at_start_of_trial'] = trial_starts.Info.str.contains('RNA=true')
trial_starts = trial_starts.drop(columns=['EC','L1','L2','N1','N2','C','Info','Event','DBLabel','HCV'])
trial_starts = trial_starts.rename(columns={'Time':'time_entered_trial'})

agent_baselines = {}
for row in trial_starts.iterrows():
    agent_data = row[1]
    agent = agent_data.Agent
    agent_baselines[agent] = agent_data
print('Agents: %d'%len(agent_baselines))

Agents: 3207


In [34]:
#trial_starts[trial_starts.Agent==254477147]

In [35]:
events2 = events[events.Agent.isin(agent_baselines)]

In [36]:
vaccinations = events2[(events2.Time <= vaccine_trial_end_date ) & (events2.Event == 'vaccinated')]
agent_vaccination_doses = vaccinations.groupby(['Agent']).size().to_dict()

In [37]:
agent_abpos_at_enrollment = {}
agent_vaccination_time = events2[(events2.Event == 'vaccinated')].groupby(['Agent'])['Time'].first().to_dict()
agent_vaccination_time = {a: float(v) for a, v in agent_vaccination_time.items()}
for event in events2.itertuples():
    if float(event.Time) < agent_vaccination_time[event.Agent]:
        agent_abpos_at_enrollment[event.Agent] = 'abneg' if event.HCV == 'susceptible' else 'abpos'

In [38]:
for agent in agent_baselines:
    agent_baselines[agent]['ABpos_at_start_of_trial'] = agent_abpos_at_enrollment[agent]
    agent_baselines[agent]['doses_received'] = agent_vaccination_doses[agent]

In [39]:
ever_infected = events2[(events2.Time <= vaccine_trial_end_date ) & (events2.Event == 'infected')]
infected_roster = ever_infected.groupby(['Agent']).size().to_dict()
#for agents with infections, count of the number of infections

In [40]:
#ever_recovered = events2[(events2.Time <= vaccine_trial_end_date ) & (events2.Event == 'recovered')]
#recovered_count = ever_recovered.groupby(['Agent']).size().to_dict()

In [41]:
#events2[events2.Agent==254477147]

In [42]:
#events2[events2.Agent == list(agent_baselines.keys())[2]]

In [43]:
#pd.unique(events2.Event)

In [44]:
#events2[events2.Agent==1359870546].iloc[0]

In [45]:
#events2[events2.Agent==300460332]

In [46]:
#the following events do not provide new information about the RNA state or the censorship, so we drop them
# if the agent becomes infected or recovered, we will retrieve it from "infected" and "recovered" events below
#  remove to simplify the parsing
# retained infectious and chronic for safety's sake
events3 = events2[~events2.Event.isin
                  (['activated','vaccinated'])]

In [47]:
#events3[events3.Agent==300460332]

In [48]:
#pd.unique(events3.Event)

In [49]:
#events2[events2.Agent==300460332]

In [50]:
agent_events = {}
longest_history = 0 #needed for padding the data
for agent in agent_baselines:
    a_events = events3[events3.Agent == agent][['Event','HCV','Info','Time']]
    a_events = a_events[a_events.Time <= vaccine_trial_end_date]
    a_events = a_events.sort_values(by="Time") #just in case
    a_events = a_events.append(pd.DataFrame([['fieldworkcompleted', '-', vaccine_trial_end_date]], columns=list(['Event','HCV', 'Time'])))
    a_events["StepsFromEnrollmentStart"] = (a_events.Time - agent_baselines[agent]["time_entered_trial"])/reporting_frequency
    agent_events[agent] = a_events
    if len(a_events[a_events.Event=='enteredfollowup']) > 0:
        longest_history = max(longest_history, a_events.Time.iloc[-1]-a_events[a_events.Event=='enteredfollowup'].Time.iloc[0])  
                          #min(vaccine_trial_end_date-a_events.Time.iloc[0], 
                          #    a_events.Time.iloc[-1]-a_events.Time.iloc[0]))

longest_history_len = longest_history/reporting_frequency

In [51]:
#a_events

In [52]:
#agent_events[300460332]

In [53]:
#print(longest_history)
#longest_history_len = longest_history/reporting_frequency
#print(longest_history_len)

#for agent, a_events in agent_events.items():
#    enrollment_start = agent_events[agent].Time.iloc[0]
#    if a_events['Time'].iloc[-1] < longest_history:
#        agent_events[agent] = agent_events[agent].append(pd.DataFrame([['padding', 'padding', longest_history]], columns=list(['Event','HCV', 'Time'])))
#    agent_events[agent].sort_values(by="Time", inplace=True)

In [54]:
#agent=list(agent_events.keys())[1]

In [55]:
#agent_events[agent]

In [56]:
#pd.unique(events3.Event)

In [57]:
#pd.unique(events3.HCV)

In [58]:
'''
produce a sequence [+,-,-,...,lost,lost,...,padding,padding,..]
the history records all infection episodes through the end of the trial or loss with padding
'''
def estimate_history(agent_id, my_events, target_length):
    h = [] #test the correct count [current_hcv]
    last_hcv_reading = None
    final_status = 'padding'
    if my_events.iloc[0].Event != 'trialstarted':
        print(f'WARNING: history for agent {agent_id} does not contain trialstarted event')
        return h, 'Unknown'
    for new_event in my_events.itertuples():
        if new_event.Event in frozenset(['enteredfollowup','infollowup','infollowup2']): #involve bleeding
            last_hcv_reading = 'RNA_positive' if 'RNA=true' in new_event.Info else 'RNA_negative'
            h = h + [last_hcv_reading]
        elif new_event.Event in frozenset(['trialabandoned', 'deactivated', 'fieldworkcompleted']):
            final_status = 'lost_to_followup'
            h = h + [final_status]
            break
        elif new_event.Event in frozenset(['trialcompleted']): #does not involve bleeding (19th event, normally)
            break
        elif new_event.Event in frozenset(['trialstarted', 'infected', 'infectious', 'recovered', 'chronic']):
            #note: we don't use these events, and so we are entirely dependent on the bleeding events
            continue
        else:
            print(f'WARNING: history for agent {agent_id} has unknown event {new_event.Event}')
    
    h = h + [final_status]*int(target_length-len(h))
    
    #pdb.set_trace()
    return h, last_hcv_reading

In [59]:
#agent_events[300460332]

In [60]:
#agent=300460332
#estimate_history(agent,
#                 agent_events[agent], 
#                 longest_history_len)

In [61]:
#estimate_history(list(agent_events.values())[2], 
#                 agent_baselines[agent]['RNA_at_start_of_trial'], 
#                 longest_history_len)

In [62]:
#list(agent_events.values())[2]

In [63]:
agent_histories = {}
for agent, a_events in agent_events.items():
    regular_history, last_hcv_reading = estimate_history(agent, 
                                                         a_events, 
                                                         longest_history_len)
    if regular_history == []: #very rare - e.g. rarely first event is "deactivated" (bug in APK)
        continue
    
    agent_baselines[agent]['notinfected_and_abpos_at_final_reading']   = \
               (agent_baselines[agent]['trial_arm'] == 'study' and last_hcv_reading == 'RNA_negative') or \
               (agent_baselines[agent]['trial_arm'] == 'placebo' and (agent in infected_roster) and last_hcv_reading == 'RNA_negative')
    #print(agent_baselines[agent]['notinfected_and_abpos_at_final_reading'])
    #print(agent_baselines[agent]['trial_arm'])
    #print(agent in infected_count)
    #print(last_hcv_reading)
    
    agent_histories[agent] = agent_baselines[agent].append(pd.Series(regular_history))

42

In [65]:
print('total agents parsed: %d'%len(agent_histories))
#events3[events3.Agent==1623552776]

total agents parsed: 3207


In [66]:
#agent_histories[1623552776]

In [67]:
#agent_histories[agent]

In [68]:
#a_events

In [69]:
#a_events

In [70]:
#agent_histories[agent]

In [71]:
#vaccine_trial_end_date

In [72]:
output_df = pd.DataFrame.from_dict(agent_histories, orient='columns')

In [73]:
#agent_0 = agent_histories.keys()[0]
#output_df.set_index(agent_baselines[agent_0].keys() + range(longest_history), inplace=True)

In [74]:
output_df = output_df.transpose()

In [75]:
output_df.to_csv(output_fpath)

In [76]:
print(f'Written: {output_fpath}')

Written: C:\Users\agutf\Sync\repos\apk-development\analysis\vaccine_cox\candidates\simphony_model_1661546710721_2weekly\instance_1\output\events.csv_history_trial721_gap14.csv


In [91]:
sample_agent = output_df.iloc[4]
for x in sample_agent: 
    print(x)
len([x for x in sample_agent if str(x).find('RNA') == 0])


1.082
1606510263
26.75890410958904
Male
NHWhite
60101
nonHR
0.0
18.0883518
1.0
0.0
1
6.027793295
0.10072007599999999
study
OutsideChicago
0.0
0.0
1.0
23.279452054794522
False
abneg
2
True
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
RNA_negative
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding
padding


44

In [78]:
#pd.read_csv(output_fpath)

-----